In [44]:
import paramiko
import configparser
import shutil
import os
import glob
import time
import configparser
import pync
import re
import paramiko
from fnmatch import fnmatch
from typing import List

In [45]:
SERIES_REGEXP = r"\w+ - S\d*.*.\w+$"
HW_REGEXP = r"\w+ - HW\d*.*.\w+$"
EXAM_REGEXP = r"\w+ - E\d*.*.\w+$"
REVISION_REGEXP = r"\w+ - R\d*.*.\w+$"
LECTURE_REGEXP = r"\w+ - (\d*|Lecture notes|SLIDES|Slides).*.\w+$"
CS_REGEXP = r"\w+ - (Summary|Tricks|Useful to know|Cheatsheet|Formulaire)*.*.\w+$"


config = configparser.ConfigParser()
config.read("config.ini")


host = config["SFTP"]["sftp_host"]
port = int(config["SFTP"]["sftp_port"])
account = config["SFTP"]["sftp_account"]
pw = config["SFTP"]["sftp_pw"]


In [46]:
def modified_path_with_regex(file_name) -> str:
    if re.search(SERIES_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[:idx + 1] + "EXERCIZES" + file_name[idx:]
    elif re.search(HW_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[:idx + 1] + "HOMEWORKS" + file_name[idx:]
    elif re.search(LECTURE_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[: idx + 1] + "LECTURES" + file_name[idx:]
    elif re.search(CS_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[: idx + 1] + "CHEATSHEETS" + file_name[idx:]
    elif re.search(EXAM_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[: idx + 1] + "EXAMS" + file_name[idx:]
    elif re.search(REVISION_REGEXP, file_name):
        idx = file_name[1:].find("/") + 1
        return file_name[: idx + 1] + "REVISIONS" + file_name[idx:]
    else:
        return file_name


def terminal_command(path: str) -> str:
    return "terminal-notifier -title 'Title' -message 'Message' -actions 'Close' -execute 'open ~/'"


In [47]:
def connect_to_SFTP(host: str, port: int, account: str, pw: str) -> paramiko.SFTPClient:
    transport = paramiko.Transport((host, port))
    transport.connect(None, account, pw)
    return paramiko.SFTPClient.from_transport(transport)

In [48]:
icloud_path = config["PATHS"]["icloud_path"]
sftp_path = config["PATHS"]["sftp_path"]
sftp_backup_path = config["PATHS"]["sftp_backup_path"]

nothing = False
pync.notify("📣 AutoMove a démarré !", title="AutoMove 🔁", actions="Close",
            execute="open \"{}\"".format(icloud_path[:-1]))


sftp = connect_to_SFTP(host, port, account, pw)
sftp.chdir(sftp_backup_path)


In [56]:
def get_files_to_upload(folder: str) -> List[str]:
    to_return = []
    folder_path = os.path.join(sftp_backup_path, folder)
    if sftp_exists(folder_path):
        for f in sftp.listdir(folder_path):
            if fnmatch(f, "*.*"):
                to_return.append(os.path.join(folder, f))
        return to_return
    else:
        return []


def sftp_exists(path: str) -> bool:
    try:
        sftp.stat(path)
        return True
    except FileNotFoundError:
        return False

In [66]:
def copy_files_to_icloud(paths: List[str]):
    try:
        for f in paths:
            new_path = modified_path_with_regex(f)
            file_name = f[f.rfind("/") + 1:]

            path_exists_on_icloud = os.path.exists(os.path.join(icloud_path, new_path))

            if not path_exists_on_icloud:
                try:
                    sftp.get(os.path.join(sftp_backup_path, f), os.path.join(icloud_path, f))

                    pync.notify(f"❓ Oups! Path inconnu\n{file_name} a été déplacé à la racine du dossier.",
                                title="AutoMove 🔁", actions="Close", execute=f'open "{os.path.join(icloud_path, f)}"')
                except FileNotFoundError:
                    pync.notify(f"❌ Oups, impossible de déplacer {file_name}.",
                                title="AutoMove 🔁", actions="Close")

            # TODO: change to sftp
            sftp.get(os.path.join(sftp_backup_path, f), os.path.join(icloud_path, new_path))

            if len(paths) == 1 and new_path != f:
                if not path_exists_on_icloud:
                    pync.notify(f"✅ ${file_name} a été copié depuis le NAS vers iCloud.", title="AutoMove 🔁", actions="Close", execute=f'open "{os.path.join(icloud_path, new_path)}"')
                else:
                    pync.notify(f"✅ ${file_name} a été mis à jour sur iCloud.", title="AutoMove 🔁", actions="Close", execute=f'open "{os.path.join(icloud_path, new_path)}"')

            elif len(paths) > 1 and new_path != f:
                pync.notify(f"✅  Bonne nouvelle ! \n{len(paths)} fichier(s) copiés sur iCloud.", title="AutoMove 🔁", actions="Close", execute=f'open "{icloud_path}"')

            elif new_path == f:
                pync.notify(f"❓ Oups! Path inconnu\n{file_name} a été déplacé à la racine du dossier.",
                            title="AutoMove 🔁", actions="Close", execute=f'open "{os.path.join(icloud_path, new_path)}"')

    except FileNotFoundError:
        pync.notify(f"❓ Oups! Soucis de path sur le NAS...\nCliquer pour debug",
                    title="AutoMove 🔁", actions="Close", execute=f'code "/Users/clementsicard/Developer/GitHub/Automove"')

In [74]:
def backup_on_NAS(paths: List[str]):
    try:
        for f in paths:
            new_path = modified_path_with_regex(f)
            file_name = f[f.rfind("/") + 1:]

            path_exists_on_NAS = sftp_exists(os.path.join(sftp_path, new_path))
            print(os.path.join(sftp_path, new_path), path_exists_on_NAS)
            # TODO: change to sftp
            sftp.rename(os.path.join(sftp_backup_path, f), os.path.join(sftp_path, new_path))

            pync.notify(f"✅  Bonne nouvelle ! \n{len(paths)} fichier(s) sauvegardé(s) sur le NAS.", title="AutoMove 🔁", actions="Close")

    except FileNotFoundError:
        pync.notify(f"❓ Oups! Soucis de path pour la sauvegarde sur le NAS...\nCliquer pour debug",
                    title="AutoMove 🔁", actions="Close", execute=f'code "/Users/clementsicard/Developer/GitHub/Automove"')

In [76]:
icloud_folders = [folder for folder in next(os.walk(icloud_path))[1] if folder[0] != "."]

files_to_upload = []

for folder in icloud_folders:
    files_to_upload_by_folder = get_files_to_upload(folder)

    if files_to_upload_by_folder:
        files_to_upload.extend(files_to_upload_by_folder)

if files_to_upload:
    nothing = False

    copy_files_to_icloud(files_to_upload)
    print("Done copying to iCloud")

    backup_on_NAS(files_to_upload)
    print("Done backing up files to NAS")

elif not nothing:
    nothing = True

['Advanced Machine Learning/Notiz 24-09-2021 (2).note', 'Advanced Machine Learning/Notiz 24-09-2021.pdf', 'Advanced Machine Learning/Notiz 24-09-2021.note', 'Advanced Machine Learning/Notiz 24-09-2021 (2).pdf']
Done copying to iCloud
/CLEMENT/BACKUP COURS EPFL-ETHZ/2021-2022/MA1/Advanced Machine Learning/Notiz 24-09-2021 (2).note False
/CLEMENT/BACKUP COURS EPFL-ETHZ/2021-2022/MA1/Advanced Machine Learning/Notiz 24-09-2021.pdf False
/CLEMENT/BACKUP COURS EPFL-ETHZ/2021-2022/MA1/Advanced Machine Learning/Notiz 24-09-2021.note False
/CLEMENT/BACKUP COURS EPFL-ETHZ/2021-2022/MA1/Advanced Machine Learning/Notiz 24-09-2021 (2).pdf False
Done backing up files to NAS
